# Titanic Survival Prediction - Análisis Exploratorio de Datos
# ================================================================

# ## 📋 Objetivo
# Realizar un análisis exploratorio completo del dataset del Titanic para:
# - Entender la estructura y calidad de los datos
# - Identificar patrones de supervivencia
# - Detectar valores faltantes y outliers
# - Generar insights para el storytelling

In [10]:
# ## 1. Importación de Librerías

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings


In [11]:
import os
import sys

path = os.path.abspath("")
ruta_proyecto = os.path.abspath(os.path.join(path, ".."))
sys.path.append(os.path.join(ruta_proyecto, ""))

from src.utils.helpers import save_current_plot

# 1. Configuración

In [ ]:
warnings.filterwarnings("ignore")
plt.style.use("seaborn-v0_8")
sns.set_palette("husl")

# Configuración de visualizaciones
plt.rcParams["figure.figsize"] = (12, 8)
plt.rcParams["font.size"] = 12

print("✅ Librerías importadas correctamente")

## 2. Carga y Primera Inspección de Datos

In [ ]:
# Cargar el dataset
df = pd.read_csv("../data/raw/titanic.csv")

print("🚢 DATASET DEL TITANIC - PRIMERA INSPECCIÓN")
print("=" * 50)
print(f"📊 Dimensiones del dataset: {df.shape}")
print(f"👥 Total de pasajeros: {len(df)}")
print(f"📈 Total de características: {len(df.columns)}")

## 3. Información General del Dataset

In [ ]:
print("\n📋 INFORMACIÓN GENERAL")
print("=" * 30)
print(df.info())

print("\n📊 ESTADÍSTICAS DESCRIPTIVAS")
print("=" * 30)
print(df.describe())

print("\n🔍 PRIMERAS 5 FILAS")
print("=" * 20)
print(df.head())

## 4. Análisis de Valores Faltantes

In [ ]:
print("\n❌ ANÁLISIS DE VALORES FALTANTES")
print("=" * 35)

missing_data = df.isnull().sum()
missing_percent = (missing_data / len(df)) * 100

missing_df = pd.DataFrame(
    {
        "Columna": missing_data.index,
        "Valores Faltantes": missing_data.values,
        "Porcentaje (%)": missing_percent.values,
    }
).sort_values("Valores Faltantes", ascending=False)

print(missing_df)

# Visualización de valores faltantes
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
missing_df_filtered = missing_df[missing_df["Valores Faltantes"] > 0]
plt.bar(
    missing_df_filtered["Columna"],
    missing_df_filtered["Valores Faltantes"],
    color="coral",
)
plt.title("Valores Faltantes por Columna")
plt.xlabel("Columnas")
plt.ylabel("Cantidad")
plt.xticks(rotation=45)

plt.subplot(1, 2, 2)
plt.bar(
    missing_df_filtered["Columna"],
    missing_df_filtered["Porcentaje (%)"],
    color="lightblue",
)
plt.title("Porcentaje de Valores Faltantes")
plt.xlabel("Columnas")
plt.ylabel("Porcentaje (%)")
plt.xticks(rotation=45)

plt.tight_layout()
save_current_plot("4-Valores Faltantes", "../results/figures/eda_plots")
plt.show()

## 5. Análisis de la Variable Objetivo (Survived)

In [ ]:
print("\n🎯 ANÁLISIS DE SUPERVIVENCIA")
print("=" * 30)

survival_counts = df["Survived"].value_counts()
survival_rate = df["Survived"].mean()

print(f"👥 Total de pasajeros: {len(df)}")
print(f"💀 Fallecidos: {survival_counts[0]} ({survival_counts[0]/len(df)*100:.1f}%)")
print(
    f"✅ Supervivientes: {survival_counts[1]} ({survival_counts[1]/len(df)*100:.1f}%)"
)
print(f"📊 Tasa de supervivencia: {survival_rate:.3f} ({survival_rate*100:.1f}%)")

# Visualización de supervivencia
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Gráfico de barras
survival_counts.plot(kind="bar", ax=axes[0], color=["crimson", "forestgreen"])
axes[0].set_title("Distribución de Supervivencia")
axes[0].set_xlabel("Supervivencia (0=No, 1=Sí)")
axes[0].set_ylabel("Número de Pasajeros")
axes[0].set_xticklabels(["Fallecidos", "Supervivientes"], rotation=0)

# Gráfico de pastel
axes[1].pie(
    survival_counts.values,
    labels=["Fallecidos", "Supervivientes"],
    colors=["crimson", "forestgreen"],
    autopct="%1.1f%%",
    startangle=90,
)
axes[1].set_title("Proporción de Supervivencia")

plt.tight_layout()
save_current_plot("5-Variable Objetivo", "../results/figures/eda_plots")
plt.show()

## 6. Análisis por Características Demográficas

### 6.1 Supervivencia por Género

In [ ]:
print("\n👫 SUPERVIVENCIA POR GÉNERO")
print("=" * 25)

gender_survival = pd.crosstab(df["Sex"], df["Survived"], margins=True)
gender_survival_pct = pd.crosstab(df["Sex"], df["Survived"], normalize="index") * 100

print("Tabla de contingencia:")
print(gender_survival)
print("\nPorcentajes por género:")
print(gender_survival_pct.round(1))

# Visualización
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
gender_survival_pct.plot(kind="bar", ax=plt.gca(), color=["crimson", "forestgreen"])
plt.title("Tasa de Supervivencia por Género")
plt.xlabel("Género")
plt.ylabel("Porcentaje (%)")
plt.legend(["Fallecidos", "Supervivientes"])
plt.xticks(rotation=0)

plt.subplot(1, 3, 2)
df.groupby("Sex")["Survived"].mean().plot(kind="bar", color="skyblue")
plt.title("Tasa de Supervivencia Promedio por Género")
plt.xlabel("Género")
plt.ylabel("Tasa de Supervivencia")
plt.xticks(rotation=0)

plt.subplot(1, 3, 3)
sns.countplot(data=df, x="Sex", hue="Survived", palette=["crimson", "forestgreen"])
plt.title("Conteo de Supervivencia por Género")
plt.xlabel("Género")
plt.ylabel("Número de Pasajeros")

plt.tight_layout()
save_current_plot("6_1-Características Demográficas", "../results/figures/eda_plots")
plt.show()

### 6.2 Supervivencia por Clase

In [ ]:
print("\n🎫 SUPERVIVENCIA POR CLASE")
print("=" * 25)

class_survival = pd.crosstab(df["Pclass"], df["Survived"], margins=True)
class_survival_pct = pd.crosstab(df["Pclass"], df["Survived"], normalize="index") * 100

print("Tabla de contingencia:")
print(class_survival)
print("\nPorcentajes por clase:")
print(class_survival_pct.round(1))

plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
class_survival_pct.plot(kind="bar", ax=plt.gca(), color=["crimson", "forestgreen"])
plt.title("Tasa de Supervivencia por Clase")
plt.xlabel("Clase")
plt.ylabel("Porcentaje (%)")
plt.legend(["Fallecidos", "Supervivientes"])
plt.xticks(rotation=0)

plt.subplot(1, 3, 2)
df.groupby("Pclass")["Survived"].mean().plot(kind="bar", color="orange")
plt.title("Tasa de Supervivencia Promedio por Clase")
plt.xlabel("Clase")
plt.ylabel("Tasa de Supervivencia")
plt.xticks(rotation=0)

plt.subplot(1, 3, 3)
sns.countplot(data=df, x="Pclass", hue="Survived", palette=["crimson", "forestgreen"])
plt.title("Conteo de Supervivencia por Clase")
plt.xlabel("Clase")
plt.ylabel("Número de Pasajeros")

plt.tight_layout()
save_current_plot("6_2-Supervivencia por Clase","../results/figures/eda_plots")
plt.show()

### 6.3 Supervivencia por Puerto de Embarque

In [ ]:
print("\n⚓ SUPERVIVENCIA POR PUERTO DE EMBARQUE")
print("=" * 35)

# Eliminar valores nulos para este análisis
df_embarked_clean = df.dropna(subset=["Embarked"])

embarked_survival = pd.crosstab(
    df_embarked_clean["Embarked"], df_embarked_clean["Survived"], margins=True
)
embarked_survival_pct = (
    pd.crosstab(
        df_embarked_clean["Embarked"], df_embarked_clean["Survived"], normalize="index"
    )
    * 100
)

print("Tabla de contingencia:")
print(embarked_survival)
print("\nPorcentajes por puerto:")
print(embarked_survival_pct.round(1))

plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
embarked_survival_pct.plot(kind="bar", ax=plt.gca(), color=["crimson", "forestgreen"])
plt.title("Tasa de Supervivencia por Puerto de Embarque")
plt.xlabel("Puerto (C=Cherbourg, Q=Queenstown, S=Southampton)")
plt.ylabel("Porcentaje (%)")
plt.legend(["Fallecidos", "Supervivientes"])
plt.xticks(rotation=0)

plt.subplot(1, 2, 2)
df_embarked_clean.groupby("Embarked")["Survived"].mean().plot(
    kind="bar", color="purple"
)
plt.title("Tasa de Supervivencia Promedio por Puerto")
plt.xlabel("Puerto")
plt.ylabel("Tasa de Supervivencia")
plt.xticks(rotation=0)

plt.tight_layout()
save_current_plot("6_3-Puerto de Embarque","../results/figures/eda_plots")
plt.show()

## 7. Análisis de Variables Numéricas

### 7.1 Distribución de Edad

In [ ]:
print("\n👶👴 ANÁLISIS DE EDAD")
print("=" * 20)

print(f"Edad promedio: {df['Age'].mean():.1f} años")
print(f"Edad mediana: {df['Age'].median():.1f} años")
print(f"Edad mínima: {df['Age'].min():.1f} años")
print(f"Edad máxima: {df['Age'].max():.1f} años")
print(f"Desviación estándar: {df['Age'].std():.1f} años")

plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
df["Age"].hist(bins=30, color="skyblue", alpha=0.7)
plt.title("Distribución de Edad")
plt.xlabel("Edad")
plt.ylabel("Frecuencia")

plt.subplot(1, 3, 2)
sns.boxplot(data=df, y="Age", color="lightcoral")
plt.title("Boxplot de Edad")

plt.subplot(1, 3, 3)
sns.boxplot(data=df, x="Survived", y="Age", palette=["crimson", "forestgreen"])
plt.title("Distribución de Edad por Supervivencia")
plt.xlabel("Supervivencia (0=No, 1=Sí)")

plt.tight_layout()
save_current_plot("7_1-Distribución de Edad", "../results/figures/eda_plots")
plt.show()

### 7.2 Distribución de Tarifa

In [ ]:
print("\n💰 ANÁLISIS DE TARIFA")
print("=" * 20)

print(f"Tarifa promedio: ${df['Fare'].mean():.2f}")
print(f"Tarifa mediana: ${df['Fare'].median():.2f}")
print(f"Tarifa mínima: ${df['Fare'].min():.2f}")
print(f"Tarifa máxima: ${df['Fare'].max():.2f}")

plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
df["Fare"].hist(bins=30, color="gold", alpha=0.7)
plt.title("Distribución de Tarifa")
plt.xlabel("Tarifa ($)")
plt.ylabel("Frecuencia")

plt.subplot(1, 3, 2)
sns.boxplot(data=df, y="Fare", color="gold")
plt.title("Boxplot de Tarifa")

plt.subplot(1, 3, 3)
sns.boxplot(data=df, x="Survived", y="Fare", palette=["crimson", "forestgreen"])
plt.title("Distribución de Tarifa por Supervivencia")
plt.xlabel("Supervivencia (0=No, 1=Sí)")

plt.tight_layout()
save_current_plot("7_2-Distribución de Tarifa", "../results/figures/eda_plots")
plt.show()

## 8. Análisis de Correlaciones

In [ ]:
print("\n🔗 MATRIZ DE CORRELACIÓN")
print("=" * 25)

# Seleccionar solo variables numéricas
numeric_cols = df.select_dtypes(include=[np.number]).columns
correlation_matrix = df[numeric_cols].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(
    correlation_matrix,
    annot=True,
    cmap="coolwarm",
    center=0,
    square=True,
    linewidths=0.5,
)
plt.title("Matriz de Correlación de Variables Numéricas")
plt.tight_layout()
save_current_plot("8-Correlaciones", "../results/figures/eda_plots")
plt.show()

# Correlaciones con la variable objetivo
print("Correlaciones con Supervivencia:")
survival_corr = correlation_matrix["Survived"].sort_values(ascending=False)
print(survival_corr)

## 9. Análisis Combinado: Género, Clase y Supervivencia

In [ ]:
print("\n👥🎫 ANÁLISIS COMBINADO: GÉNERO Y CLASE")
print("=" * 40)

# Crear tabla cruzada
combined_analysis = (
    df.groupby(["Sex", "Pclass"])["Survived"].agg(["count", "sum", "mean"]).round(3)
)
combined_analysis.columns = ["Total", "Supervivientes", "Tasa_Supervivencia"]
print(combined_analysis)

# Visualización
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
sns.barplot(
    data=df, x="Pclass", y="Survived", hue="Sex", palette=["lightcoral", "lightblue"]
)
plt.title("Tasa de Supervivencia por Clase y Género")
plt.xlabel("Clase")
plt.ylabel("Tasa de Supervivencia")

plt.subplot(1, 3, 2)
pivot_table = df.pivot_table(
    values="Survived", index="Sex", columns="Pclass", aggfunc="mean"
)
sns.heatmap(
    pivot_table, annot=True, cmap="RdYlGn", cbar_kws={"label": "Tasa de Supervivencia"}
)
plt.title("Heatmap: Supervivencia por Género y Clase")

plt.subplot(1, 3, 3)
sns.countplot(data=df, x="Pclass", hue="Sex", palette=["lightcoral", "lightblue"])
plt.title("Distribución de Pasajeros por Clase y Género")
plt.xlabel("Clase")
plt.ylabel("Número de Pasajeros")

plt.tight_layout()
save_current_plot("9-Análisis Combinado", "../results/figures/eda_plots")
plt.show()